In [1]:
import pandas as pd
from helpers import *
from utils import *
from sklearn.preprocessing import OrdinalEncoder

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [2]:
exclude_column = [
    'Scrape Date',
    'Added on',
    'Name',
    'Club ID',
    'League ID',
    'Nation ID (nationality)',
    'All versions resource ID',
    'GAMES(PS,Xbox,PC)',
    'GOALS(PS,Xbox,PC)',
    'ASSISTS(PS,Xbox,PC)',
    'YELLOW(PS,Xbox,PC)',
    'RED(PS,Xbox,PC)',
    'TOP 3 CHEM(PS,Xbox,PC)',
    'PC dialy prices',
    'Xbox dialy prices',
    'PS dialy prices',
    'Scraped Url'
]

In [3]:
dataset_2022 = pd.read_csv(Path(data_root, "Database_2022.csv"))

pc_prices = dataset_2022[["PC dialy prices"]]
pc_prices["PC dialy prices"] = pc_prices["PC dialy prices"].apply(lambda x: split_price_history(x))
pc_prices = pc_prices.explode("PC dialy prices")
price_history = pc_prices["PC dialy prices"].apply(pd.Series)
price_history.columns = ["date", "price"]
pc_prices = pd.concat([pc_prices, price_history], axis=1)

pc_players = dataset_2022[["Data-version", "Rating", "Name"]]
pc_players = pd.merge(pc_players, pc_prices, left_index=True, right_index=True)

train_columns = [x for x in dataset_2022.columns if x not in exclude_column]
train_data = dataset_2022[train_columns]

pc_games = dataset_2022['GAMES(PS,Xbox,PC)'].apply(lambda x: float_convert(x, 0))
pc_goals = dataset_2022['GOALS(PS,Xbox,PC)'].apply(lambda x: float_convert(x, 0))
pc_assists = dataset_2022['ASSISTS(PS,Xbox,PC)'].apply(lambda x: float_convert(x, 0))
train_data = pd.merge(train_data, pc_games, left_index=True, right_index=True)
train_data = pd.merge(train_data, pc_players[["price"]], left_index=True, right_index=True)
train_data.fillna(0, inplace=True)

oe = OrdinalEncoder()
train_data[["Data-version", "Position"]] = oe.fit_transform(train_data[["Data-version", "Position"]])

save_object(oe, "oe")
train_data = train_data.loc[train_data.price > 0]
train_data.drop_duplicates(inplace=True)
print(train_data.head(10))
print(train_data.shape)

/tmp/ipykernel_6461/4191057277.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_prices["PC dialy prices"] = pc_prices["PC dialy prices"].apply(lambda x: split_price_history(x))


   Data-version Rating  Position Player ID PACE diving Handling Kicking  \
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   
4           9.0     95       0.0    237074   88      0        0       0   

  Reflexes Speed  ... Strength Aggression Physical Mental Skill Chem. Style  \
4        0     0  ...       74         76        0    685   404        1182   
4        0     0

In [4]:
train_data.columns

Index(['Data-version', 'Rating', 'Position', 'Player ID', 'PACE', 'diving',
       'Handling', 'Kicking', 'Reflexes', 'Speed', 'Positioning', 'SHOOTING',
       'PASSING', 'PHYSICALITY', 'DEFENDING', 'DRIBBLING', 'Acceleration',
       'Sprint Speed', 'Positioning.1', 'Finishing', 'Shot Power',
       'Long Shots', 'Volleys', 'Penalties', 'Vision', 'Crossing',
       'FK. Accuracy', 'Short Passing', 'Long Passing', 'Curve', 'Agility',
       'Balance', 'Reactions', 'Ball Control', 'Sub-Dribbling', 'Composure',
       'Interceptions', 'Heading Accuracy', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Jumping', 'Stamina', 'Strength', 'Aggression',
       'Physical', 'Mental', 'Skill', 'Chem. Style', 'Base stats',
       'In game stats', 'GAMES(PS,Xbox,PC)', 'price'],
      dtype='object')

In [5]:
train_data = train_data[["Data-version", "Rating", "Player ID", "price"]]
train_data.head(55)

,Data-version,Rating,Player ID,price
4,9.0,95,237074,2700000.0
4,9.0,95,237074,2642500.0
4,9.0,95,237074,2785000.0
4,9.0,95,237074,2499000.0
4,9.0,95,237074,2670000.0
4,9.0,95,237074,2645000.0
4,9.0,95,237074,2527500.0
4,9.0,95,237074,3005000.0
4,9.0,95,237074,2625000.0
4,9.0,95,237074,2712000.0


In [6]:
group_data = train_data.groupby('Player ID').agg(list).reset_index()
group_data[["Player ID"]].values

array([['1040'],
       ['104389'],
       ['1075'],
       ...,
       ['84127176'],
       ['84132271'],
       ['942']], dtype=object)

In [11]:
def shift(x):
    x["price_new"] = x["price"].shift(-1)
    return x

shifted_data = train_data.iloc[0:100].groupby('Player ID').apply(shift)
shifted_data["diff"] = shifted_data["price"].iloc[0:100] - shifted_data["price_new"].iloc[0:100]
shifted_data.dropna(inplace=True)
shifted_data.iloc[48:100]

,Data-version,Rating,Player ID,price,price_new,diff
4,9.0,95,237074,2400500.0,2400000.0,500.0
4,9.0,95,237074,2400000.0,2975500.0,-575500.0
4,9.0,95,237074,2975500.0,3150000.0,-174500.0
4,9.0,95,237074,3150000.0,2687500.0,462500.0
5,9.0,95,237068,6375000.0,6300000.0,75000.0
5,9.0,95,237068,6300000.0,6290000.0,10000.0
5,9.0,95,237068,6290000.0,6199000.0,91000.0
5,9.0,95,237068,6199000.0,6244500.0,-45500.0
5,9.0,95,237068,6244500.0,6200000.0,44500.0
5,9.0,95,237068,6200000.0,6490000.0,-290000.0


In [16]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=2)
for i, (train_index, test_index) in enumerate(tscv.split(shifted_data)):
    print(i)
    print(train_index, test_index)
#     train, test = shifted_data.loc[train_index], shifted_data.loc[test_index]


0
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32] [33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56
 57 58 59 60 61 62 63 64]
1
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64] [65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88
 89 90 91 92 93 94 95 96]
